<a href="https://colab.research.google.com/github/preethi-raddi/feature-engineering-ASAG/blob/main/paper1_code2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages(c("caret", "e1071", "randomForest", "tm","text2vec"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘sparsevctrs’, ‘timeDate’, ‘MatrixExtra’, ‘float’, ‘RhpcBLASctl’, ‘RcppArmadillo’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’, ‘proxy’, ‘NLP’, ‘slam’, ‘BH’, ‘rsparse’, ‘mlapi’, ‘lgr’




In [2]:
# Loading necessary libraries
library(tm)
library(text2vec)
library(stringr)

Loading required package: NLP



In [3]:
# Sample Dataset
data <- read.csv("/content/data_da2.csv")

head(data,10)

,id,question,desired_answer,student_answer,score_me,score_other,score_avg
,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>
1,3.2,What is the scope of global variables?,File scope.,Any functions that follows their deceleration that are in the source code file are allowed to have access the variables. Also global scope variables retrain their value throughout the execution of the program.,5,5,5.0
2,3.2,What is the scope of global variables?,File scope.,they can be accessed by any C++ file anywhere.,0,5,2.5
3,3.2,What is the scope of global variables?,File scope.,Global variables have file scope.,5,5,5.0
4,3.2,What is the scope of global variables?,File scope.,The scope of global variables are created by placing variable declarations outside any class or function definition. Global variables retain their values throughout the execution of the program and they can be referenced by any function that follows their declarations or definitions in the source file.,0,5,2.5
5,3.2,What is the scope of global variables?,File scope.,A global variable is an identifier that retains it's value throughout the execution of the program. It has file scope.,5,5,5.0
6,3.2,What is the scope of global variables?,File scope.,Global Variables can be used in any function as long as the appropriate .h file that holds the variable is included,3,5,4.0
7,3.2,What is the scope of global variables?,File scope.,They can be accessed by any part of the program. It can be referenced by any function that follows the declarations or definitions in the source file.,5,5,5.0
8,3.2,What is the scope of global variables?,File scope.,can be accesed by any classes that have and object of that variables class in it,1,5,3.0
9,3.2,What is the scope of global variables?,File scope.,Global variables have file-scope. The variable is know to all functions throught the file,5,5,5.0


In [ ]:
# Preprocessing Function for Text
preprocess_text <- function(text) {
  corpus <- Corpus(VectorSource(text))
  corpus <- tm_map(corpus, content_transformer(tolower))
  corpus <- tm_map(corpus, removePunctuation)
  corpus <- tm_map(corpus, removeWords, stopwords("en"))
  corpus <- tm_map(corpus, stripWhitespace)
  return(unlist(strsplit(as.character(corpus[[1]]$content), " ")))
}

# Apply Preprocessing
data$StudentKeywords <- lapply(data$student_answer, preprocess_text)
data$ModelKeywords <- list(preprocess_text(data$desired_answer[1]))

In [6]:
# Function to Compute Jaccard Similarity
jaccard_similarity <- function(set1, set2) {
  intersection <- length(intersect(set1, set2))
  union <- length(unique(c(set1, set2)))
  return(intersection / union)
}

In [7]:
# Compute Keyword Similarities
data$KeywordSimilarity <- sapply(data$StudentKeywords, function(student) {
  jaccard_similarity(student, unlist(data$ModelKeywords))
})

In [8]:
# Identify High-Similarity Keywords
threshold <- 0.5  # Set threshold for high similarity
high_similarity_keywords <- list()
for (i in 1:nrow(data)) {
  if (data$KeywordSimilarity[i] >= threshold) {
    high_similarity_keywords <- unique(c(high_similarity_keywords, data$StudentKeywords[[i]]))
  }
}

In [9]:
# Mutate Model Answer Key
updated_model_keywords <- unique(c(unlist(data$ModelKeywords), high_similarity_keywords))

print(updated_model_keywords)

[[1]]
[1] "file"

[[2]]
[1] "scope"

[[3]]
[1] "global"

[[4]]
[1] "variables"

